# 이미지 데이터 셋을 이용한 CNN Modeling
> # Github
## Dogs and Cats Image_Data
* Train_Data : 2000(1000_Dogs, 1000_Cats)
* Valid_Data : 1000(500_Dogs, 500_Cats)
* Test_Data : 1000(500_Dogs, 500_Cats)

In [1]:
import warnings
warnings.filterwarnings('ignore')

* Import TensorFlow

In [2]:
import tensorflow as tf

tf.__version__

'2.10.0'

# I. wget From Github
* 'dogs_and_cats_small.zip' 파일 다운로드

In [ ]:
# !wget https://raw.githubusercontent.com/rusita-ai/pyData/master/dogs_and_cats_small.zip

--2023-01-26 01:28:20--  https://raw.githubusercontent.com/rusita-ai/pyData/master/dogs_and_cats_small.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 90618980 (86M) [application/zip]
Saving to: ‘dogs_and_cats_small.zip’

dogs_and_cats_small 100%[===================>]  86.42M  68.6MB/s    in 1.3s    

2023-01-26 01:28:26 (68.6 MB/s) - ‘dogs_and_cats_small.zip’ saved [90618980/90618980]



* 다운로드 결과 확인

In [ ]:
# !ls -l 

total 88500
-rw-r--r-- 1 root root 90618980 Jan 26 01:28 dogs_and_cats_small.zip
drwxr-xr-x 1 root root     4096 Jan 24 14:38 sample_data


# II. Data Preprocessing

> ## 1) Unzip 'dogs_and_cats_small.zip'

In [ ]:
# !unzip -q /content/dogs_and_cats_small.zip

* 학습 데이터 확인

In [ ]:
# !ls -l

total 88512
-rw-r--r-- 1 root root 90618980 Jan 26 01:28 dogs_and_cats_small.zip
drwxr-xr-x 1 root root     4096 Jan 24 14:38 sample_data
drwxr-xr-x 4 root root     4096 Jan 26 01:28 test
drwxr-xr-x 4 root root     4096 Jan 26 01:28 train
drwxr-xr-x 4 root root     4096 Jan 26 01:28 validation


> ## 2) Image_File Directory Setting

* train_dir
* valid_dir
* test_dir

In [6]:
!cd

C:\Users\SBAUser\PycharmProjects\SESAC_AIoT\example\15_DL\05_Convolutional_Neural_Network_CNN


In [7]:
train_dir = './../../../secret/example_data/15_DL/dogs_and_cats_small/train'
valid_dir = './../../../secret/example_data/15_DL/dogs_and_cats_small/validation'
test_dir  = './../../../secret/example_data/15_DL/dogs_and_cats_small/test'

> ## 3) ImageDataGenerator( ) & flow_from_directory( )

* Normalization
 * ImageDataGenerator( )
* Resizing & Generator
 * flow_from_directory( )

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255) # 0~1 scaling
valid_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(
                  train_dir,
                  target_size = (150, 150),
                  batch_size = 20,
                  class_mode = 'binary')

valid_generator = valid_datagen.flow_from_directory(
                  valid_dir,
                  target_size = (150, 150),
                  batch_size = 20,
                  class_mode = 'binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


> ## 4) Test train_generator

In [9]:
for data_batch, labels_batch in train_generator:
    print('배치 데이터 크기:', data_batch.shape)
    print('배치 레이블 크기:', labels_batch.shape)
    break

배치 데이터 크기: (20, 150, 150, 3)
배치 레이블 크기: (20,)


In [10]:
labels_batch

array([0., 1., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1.,
       0., 1., 1.], dtype=float32)

# III. CNN Keras Modeling

> ## 1) Model Define

* Feature Extraction & Classification

In [11]:
from tensorflow.keras import layers
from tensorflow.keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = (150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(512, activation = 'relu'))
model.add(layers.Dense(1, activation = 'sigmoid'))

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 17, 17, 128)      0

> ## 2) Model Compile

* 모델 학습방법 설정

In [13]:
model.compile(loss = 'binary_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

> ## 3) Model Fit

* 모델 학습 수행
 * 약 10분

In [14]:
# %%time

Hist_dandc = model.fit(train_generator,
                       steps_per_epoch = 100,
                       epochs = 60,
                       validation_data = valid_generator,
                       validation_steps = 50)

Epoch 1/60
100/100 [==============================] - 12s 40ms/step - loss: 0.7035 - accuracy: 0.4980 - val_loss: 0.6866 - val_accuracy: 0.5060
Epoch 2/60
100/100 [==============================] - 3s 25ms/step - loss: 0.6924 - accuracy: 0.5110 - val_loss: 0.6727 - val_accuracy: 0.6230
Epoch 3/60
100/100 [==============================] - 3s 25ms/step - loss: 0.6836 - accuracy: 0.5745 - val_loss: 0.6601 - val_accuracy: 0.6150
Epoch 4/60
100/100 [==============================] - 3s 25ms/step - loss: 0.6518 - accuracy: 0.6195 - val_loss: 0.6822 - val_accuracy: 0.5970
Epoch 5/60
100/100 [==============================] - 2s 25ms/step - loss: 0.6204 - accuracy: 0.6665 - val_loss: 0.6124 - val_accuracy: 0.6540
Epoch 6/60
100/100 [==============================] - 2s 25ms/step - loss: 0.5785 - accuracy: 0.6995 - val_loss: 0.6096 - val_accuracy: 0.6530
Epoch 7/60
100/100 [==============================] - 2s 25ms/step - loss: 0.5317 - accuracy: 0.7335 - val_loss: 0.5682 - val_accuracy: 0.700

Epoch 57/60
100/100 [==============================] - 3s 26ms/step - loss: 2.1586e-06 - accuracy: 1.0000 - val_loss: 2.7737 - val_accuracy: 0.7430
Epoch 58/60
100/100 [==============================] - 3s 25ms/step - loss: 2.0080e-06 - accuracy: 1.0000 - val_loss: 2.7864 - val_accuracy: 0.7450
Epoch 59/60
100/100 [==============================] - 3s 26ms/step - loss: 1.8254e-06 - accuracy: 1.0000 - val_loss: 2.7968 - val_accuracy: 0.7430
Epoch 60/60
100/100 [==============================] - 3s 26ms/step - loss: 1.7254e-06 - accuracy: 1.0000 - val_loss: 2.8081 - val_accuracy: 0.7450


> ## 4) 학습 결과 시각화

In [15]:
import matplotlib.pyplot as plt

epochs = range(1, len(Hist_dandc.history['loss']) + 1)

plt.figure(figsize = (15, 5))

plt.subplot(1, 2, 1)
plt.plot(epochs, Hist_dandc.history['loss'], 'b-')
plt.plot(epochs, Hist_dandc.history['val_loss'], 'r--')
plt.title('Training & Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['Training Loss', 'Validation Loss'])
plt.grid()

plt.subplot(1, 2, 2)
plt.plot(epochs, Hist_dandc.history['accuracy'], 'b-')
plt.plot(epochs, Hist_dandc.history['val_accuracy'], 'r--')
plt.title('Training & Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['Training Accuracy', 'Validation Accuracy'])
plt.grid()

plt.show()

> ## 5) Model Evaluate

* test_generator

In [16]:
test_datagen = ImageDataGenerator(rescale = 1./255)

test_generator = test_datagen.flow_from_directory(
                 test_dir,
                 target_size = (150, 150),
                 batch_size = 20,
                 class_mode = 'binary')

Found 1000 images belonging to 2 classes.


* Loss & Accuracy

In [17]:
loss, accuracy = model.evaluate(test_generator,
                                steps = 50, 
                                verbose = 0)

print('Loss = {:.5f}'.format(loss))
print('Accuracy = {:.5f}'.format(accuracy))

Loss = 3.10447
Accuracy = 0.73000


# IV. Model Save & Load to Google Drive

> ## 1) Google Drive Mount

* Mount Google Drive

In [ ]:
# from google.colab import drive

# drive.mount('/content/drive')

Mounted at /content/drive


* Make 'models' Directory

In [ ]:
# !mkdir -p '/content/drive/My Drive/Colab Notebooks/models'

> ## 2) Model Save

In [22]:
# model.save('/content/drive/My Drive/Colab Notebooks/models/002_dogs_and_cats_small.h5')
model.save('dogs_and_cats_small.h5')

In [ ]:
# !ls -l /content/drive/My\ Drive/Colab\ Notebooks/models

total 40527
-rw------- 1 root root 41499528 Jan 26 01:38 002_dogs_and_cats_small.h5


> ## 3) Model Load

In [23]:
from tensorflow.keras.models import load_model

# model_small = load_model('/content/drive/My Drive/Colab Notebooks/models/002_dogs_and_cats_small.h5')
model_small = load_model('dogs_and_cats_small.h5')

In [24]:
loss, accuracy = model_small.evaluate(test_generator,
                                      steps = 50, 
                                      verbose = 0)

print('Loss = {:.5f}'.format(loss))
print('Accuracy = {:.5f}'.format(accuracy))

Loss = 3.10447
Accuracy = 0.73000


# 
# 
# 
# The End
# 
# 
# 